<a href="https://colab.research.google.com/github/hfathie/hfvSPH_on_GPU/blob/main/acc_sph_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%%writefile test.cu
#include <iostream>  // iostream, fstream, cmath, string, vector, sstream.
#include <fstream>
#include <cmath>
#include <string>
#include <vector>
#include <sstream>
#include "myCppSPHLibs.h"
using namespace std;

const int N = 131504;
const float my_pi = 3.141592f;
const float visc_alpha = 1.0f;


int main(){

  // Reading Hydra file.
  string fname = "Hydra_130k.csv";

  vector<vector<string>> content;
  vector<string> row;
  string line, word;
  
  fstream file (fname, ios::in);
  if(file.is_open())
  {
  while(getline(file, line))
  {
  row.clear();
  
  stringstream str(line);
  
  while(getline(str, word, ','))
  row.push_back(word);
  content.push_back(row);
  }
  }
  else
  cout<<"Could not open the file\n";

  float *x,*y,*z, *vx,*vy,*vz, *rho, *P, *c, *h, *mass, *divV, *curlV;
  float *d_x,*d_y,*d_z, *d_vx,*d_vy,*d_vz, *d_rho, *d_P, *d_c, *d_h;
  float *d_mass, *d_divV, *d_curlV, *ax, *ay, *az, *d_ax, *d_ay, *d_az;

  x = new float[N];
  y = new float[N];
  z = new float[N];

  vx = new float[N];
  vy = new float[N];
  vz = new float[N];

  ax = new float[N];
  ay = new float[N];
  az = new float[N];

  rho = new float[N];
  P = new float[N];
  c = new float[N];
  h = new float[N];
  mass = new float[N];
  divV = new float[N];
  curlV = new float[N];

  // 0  1  2  3   4   5    6   7  8  9  10
  // x, y, z, vx, vy, vz, rho, P, c, h, m.

  for(int i=0; i<N; i++){

    x[i] = stof(content[i][0]);
    y[i] = stof(content[i][1]);
    z[i] = stof(content[i][2]);

    vx[i] = stof(content[i][3]);
    vy[i] = stof(content[i][4]);
    vz[i] = stof(content[i][5]);

    rho[i] = stof(content[i][6]);
    P[i] = stof(content[i][7]);
    c[i] = stof(content[i][8]);
    h[i] = stof(content[i][9]);
    mass[i] = stof(content[i][10]);

    divV[i] = stof(content[i][11]);
    curlV[i] = stof(content[i][12]);

    ax[i] = 0.0f;
    ay[i] = 0.0f;
    az[i] = 0.0f;

  }

  cudaMalloc(&d_x, N*sizeof(float));
  cudaMalloc(&d_y, N*sizeof(float));
  cudaMalloc(&d_z, N*sizeof(float));

  cudaMalloc(&d_vx, N*sizeof(float));
  cudaMalloc(&d_vy, N*sizeof(float));
  cudaMalloc(&d_vz, N*sizeof(float));

  cudaMalloc(&d_ax, N*sizeof(float));
  cudaMalloc(&d_ay, N*sizeof(float));
  cudaMalloc(&d_az, N*sizeof(float));

  cudaMalloc(&d_rho, N*sizeof(float));
  cudaMalloc(&d_P, N*sizeof(float));
  cudaMalloc(&d_c, N*sizeof(float));
  cudaMalloc(&d_h, N*sizeof(float));
  cudaMalloc(&d_mass, N*sizeof(float));
  cudaMalloc(&d_divV, N*sizeof(float));
  cudaMalloc(&d_curlV, N*sizeof(float));

  // Copy from Host to Device.
  cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_z, z, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_vx, vx, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_vy, vy, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_vz, vz, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_ax, ax, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_ay, ay, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_az, az, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_rho, rho, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_P, P, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_c, c, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_h, h, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_mass, mass, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_divV, divV, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_curlV, curlV, N*sizeof(float), cudaMemcpyHostToDevice);

  // Launching the kernel on GPU
  int blockSize = 256; // number of threads in a block
  int gridSize = (N + blockSize - 1) / blockSize; // Number of blocks in a grid

  acc_sph<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_vx, d_vy, d_vz, d_h, d_c, d_rho,
                                   d_divV, d_curlV, d_mass, d_P, d_ax, d_ay, d_az,
                                   my_pi, visc_alpha, N);

  // Wait for the GPU to finish before accessing the Host
  cudaDeviceSynchronize();

  // Copy from Device to Host.
  cudaMemcpy(ax, d_ax, N*sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(ay, d_ay, N*sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(az, d_az, N*sizeof(float), cudaMemcpyDeviceToHost);

  // visual check.
  for(int i = 0; i < 10; i++){
    cout << ax[i] << "," << ay[i] << "," << az[i] << endl;
  }

  // Free memory.
  cudaFree(d_x);
  cudaFree(d_y);
  cudaFree(d_z);

  cudaFree(d_vx);
  cudaFree(d_vy);
  cudaFree(d_vz);

  cudaFree(d_ax);
  cudaFree(d_ay);
  cudaFree(d_az);

  cudaFree(d_rho);
  cudaFree(d_P);
  cudaFree(d_c);
  cudaFree(d_h);
  cudaFree(d_mass);
  cudaFree(d_divV);
  cudaFree(d_curlV);

  delete[] x;
  delete[] y;
  delete[] z;

  delete[] vx;
  delete[] vy;
  delete[] vz;

  delete[] ax;
  delete[] ay;
  delete[] az;

  delete[] rho;
  delete[] P;
  delete[] c;
  delete[] h;
  delete[] mass;
  delete[] divV;
  delete[] curlV;

}

Overwriting test.cu


In [10]:
%%shell
nvcc test.cu -o test

In [11]:
%%shell
./test

0.703798,-3.19147,5.94139
8.00734,-1.25612,-12.0063
-3.42585,-9.84311,1.03338
12.6441,-0.0720719,-0.851646
4.04376,5.91137,-11.0488
1.46846,5.80696,-4.86355
9.8988,5.7009,4.4007
-1.01352,-4.28169,0.533488
2.38816,-1.51075,-6.8932
-0.651897,3.75301,0.429989


In [12]:
%%shell
nvprof ./test

==307== NVPROF is profiling process 307, command: ./test
0.703798,-3.19147,5.94139
8.00734,-1.25612,-12.0063
-3.42585,-9.84311,1.03338
12.6441,-0.0720719,-0.851646
4.04376,5.91137,-11.0488
1.46846,5.80696,-4.86355
9.8988,5.7009,4.4007
-1.01352,-4.28169,0.533488
2.38816,-1.51075,-6.8932
-0.651897,3.75301,0.429989
==307== Profiling application: ./test
==307== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.74%  328.12ms         1  328.12ms  328.12ms  328.12ms  acc_sph(float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float*, float, float, int)
                    0.22%  727.09us        16  45.443us  45.087us  46.016us  [CUDA memcpy HtoD]
                    0.04%  130.91us         3  43.636us  41.887us  47.136us  [CUDA memcpy DtoH]
      API calls:   50.71%  328.16ms         1  328.16ms  328.16ms  328.16ms  cudaDeviceSynchronize
                 